# Exercise 4

Work on this before the next lecture on 26 April. We will talk about questions, comments, and solutions during the exercise after the third lecture.

Please do form study groups! When you do, make sure you can explain everything in your own words, do not simply copy&paste from others.

The solutions to a lot of these problems can probably be found with Google. Please don't. You will not learn a lot by copy&pasting from the internet.

If you want to get credit/examination on this course please upload your work to your GitHub repository for this course before the next lecture starts and post a link to your repository in [this thread](https://github.com/wildtreetech/advanced-computing-2018/issues/8). If you worked on things together with others please add their names to the notebook so we can see who formed groups.

The overall idea of this exercise is to get you using and building convolutional neural networks.

## Question 1

In the last exercise you built a neural network that can classify fashion items using only densely connected layers.

Build on this by using convolutions, pooling, dropout, batch norm, etc in your neural network. Can you outperform your densely connected network?

Start with a small network and a fraction of the data to check if you hooked everything up correctly. Don't go overboard with the size of the network either as even small networks take quite a while to train.

(If you want to experiment with a free GPU checkout https://kaggle.com/kernels .)

In [2]:
# your code here
from sklearn.model_selection import train_test_split
from keras import utils
from keras.models import Model
from keras.layers import Input, Dense, Activation, Flatten, Conv2D, MaxPooling2D
from keras.datasets import fashion_mnist
import numpy as np

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

X_train = X_train[:, :, :, np.newaxis]
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size=10000,
                                                  random_state=42)


num_classes = 10
y_train = utils.to_categorical(y_train, num_classes)
y_val = utils.to_categorical(y_val, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

# This returns a tensor to represent the input
inputs = Input(shape=(28,28,1))

# a layer instance is callable on a tensor, and returns a tensor
x = Conv2D(100, 3, activation="elu")(inputs)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(20, 5, activation="elu")(x)
x = Flatten()(x)
x = Dense(10)(x)
# to find out more about activations check the keras documentation
predictions = Activation("softmax")(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'],
              )

# to fit the model uncomment this line, experiment with the various settings
model.fit(X_train, y_train, epochs=10, verbose=True)
print(X_train.shape, y_train.shape)


(50000, 28, 28, 1) (50000, 10)
Epoch 1/10
50000/50000 [==============================] - 58s 1ms/step - loss: 0.7237 - acc: 0.7407
Epoch 2/10
50000/50000 [==============================] - 56s 1ms/step - loss: 0.4851 - acc: 0.8277
Epoch 3/10
50000/50000 [==============================] - 57s 1ms/step - loss: 0.4363 - acc: 0.8455
Epoch 4/10
50000/50000 [==============================] - 59s 1ms/step - loss: 0.4052 - acc: 0.8563
Epoch 5/10
50000/50000 [==============================] - 57s 1ms/step - loss: 0.3840 - acc: 0.8638
Epoch 6/10
50000/50000 [==============================] - 57s 1ms/step - loss: 0.3663 - acc: 0.8697
Epoch 7/10
50000/50000 [==============================] - 57s 1ms/step - loss: 0.3526 - acc: 0.8761
Epoch 8/10
50000/50000 [==============================] - 56s 1ms/step - loss: 0.3411 - acc: 0.8801
Epoch 9/10
50000/50000 [==============================] - 55s 1ms/step - loss: 0.3318 - acc: 0.8832
Epoch 10/10
50000/50000 [==============================] - 54s 1ms/st

## Question 2

For most real world applications we do not have enough labelled images to train a large neural network from scratch. Instead we can use a pre-trained network as a feature transformer and train a smaller model (or even just a logistic regression) on the output of the pre-trained network.

There are several pretrained networks available as part of keras: https://keras.io/applications/. The documentation usually gives some information or links about each network.

The documentation also contains snippets on how to use a pre-trained network as feature transformer ("Extract features with VGG16"). You should be able to generalise from that example using VGG16 to approximately any of the networks available there.

One important thing to not forget is that you need to preprocess your images before feeding them into a pretrained network. Keras provides the functions to do that as well, use them :) You might also need to resize your images first.

The task for this question is to build a classifier that can tell road bikes from mountain bikes. Start with using a pre-trained network as feature transformer and logistic regression as classifier on the output of the pretrained network. Once this works you can experiment with extracting features from earlier layers of the pre-trained network, compare your performance to a small network trained from scratch, try to beat your neural net by extracting features by hand and feeding them to a random forest, increasing your dataset size by [augmenting the data](https://keras.io/preprocessing/image/), etc.

The dataset containing about 100 labelled images for each road and mountain bikes is here: https://github.com/wildtreetech/advanced-computing-2018/blob/master/data/road-and-mountain-bikes.zip

In [7]:
from keras import applications
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from glob import glob

from urllib.request import urlretrieve
from zipfile import ZipFile
from os import path

zip_name = "tmp.zip"
if not path.isfile(zip_name):
    urlretrieve("https://github.com/wildtreetech/advanced-computing-2018/raw/master/data/road-and-mountain-bikes.zip",
                zip_name)

if not path.isdirectory("bikes"):
    zip_file = ZipFile("tmp.zip")
    zip_file.extractall("./")
    zip_file.close()

# for example load the VGG16 network
model = applications.VGG16(include_top=False,
                           weights='imagenet')

def read_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

# Process data
mountain_bikes = glob("bikes/mountain_bikes/*.jpg")
road_bikes = glob("bikes/road_bikes/*.jpg")

X = [*mountain_bikes, *road_bikes]
y = [*[0]*len(mountain_bikes), *[1]*len(road_bikes)]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                  test_size=0.8,
                                                  random_state=42)

def predict(X_val):
    N = len(X_val)
    for i in range(N):
        img = read_image(X_val[i])
        tmp = model.predict(img)
        X_val[i] = np.array(tmp).reshape(-1)
    return np.array(X_val)

X_train = predict(X_train)
X_test = predict(X_test)

print(X_train.shape, X_test.shape)

AttributeError: module 'posixpath' has no attribute 'isdirectory'

In [96]:
from keras.layers import Input, Dense, Activation
from keras.models import Model

# Do Machine learning
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

print("Score random forest:", rfc.score(X_test, y_test))

print(X_train.shape)
# This returns a tensor to represent the input
inputs = Input(shape=(25088,))

x = Activation('tanh')(inputs)
# a layer instance is callable on a tensor, and returns a tensor
x = Dense(10)(x)
x = Activation("elu")(x)
# to find out more about activations check the keras documentation
predictions = Activation('softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              )

# to fit the model uncomment this line, experiment with the various settings
model.fit(X_train, y_train, epochs=10, verbose=False)

print("Score neural network:", model.evaluate(X_test, y_test)[1])
print("Neural network get a score of 1.0, therefore I will not be able to beat it")

Score random forest: 0.9940828402366864
(42, 25088)
169/169 [==============================] - 0s 2ms/step
Score neural network: [0.05319205918431987, 1.0]
Neural network get a score of 1.0, therefore I will not be able to beat it


## Question 3

Think about what project you want to do. What makes a good project? It should use some of what you learnt in this class, there should be labelled data available already, and it should be something you are interested in.

You will have to write a short report on what you did. To write an interesting report you need to tell a story, not just first I did A, then I did B, then I did X and finally D.

It also has to go a bit beyond simply training a classifier or regression model.

An example based on the bike images from the previous question:

A local bike shop wants to keep an eye on sales of bikes on ebay. They specialise in road bikes so they want to be able to filter out all adverts for mountain bikes. They have found that people writing ebay adverts are not very good at correctly labelling their adverts. Can they use machine-learning to help classify adverts?

We investigate labelling adverts based on the image in the advert and study different trade offs in misclassifying bikes. The network was trained on 100 images from a catalog which show bikes on a white background. We compare the performance of the network on the training data and a small set of hand labelled images of bikes in the wild.